# ChainerによるResidual Network (ResNet)

## 本チュートリアルではchainerを利用してニューラルネットワークの実装を確認，学習および評価を行います．　環境としてはGoogle が提供する Google Colaboratory上でおこないます． GPU上で処理を行うため，colaboratoryの[ランタイム]->[ランタイムのタイプを変更]からハードウェアアクセラレータをGPUにしてください．

Goolge Colaboratory上にChainerとCupyをインストールします．

In [ ]:
!curl https://colab.chainer.org/install | sh -
!pip install 'chainercv'

Chainerでニューラルネットワークを学習するために必要なモジュールや関数をインポートします．また，data augmentationを行うために，画像変換にopencvを利用します．opencvのモジュールであるcv2も合わせてインポートします．

In [ ]:
import numpy as np
import chainer
from chainer import cuda,optimizers, serializers, Chain, Variable
import chainer.functions as F
import chainer.links as L
from chainer.datasets import TransformDataset
from chainercv import transforms
from functools import partial
import cv2


GPUが利用できるか確認します．

In [ ]:
print('GPU availability:', chainer.cuda.available)
print('cuDNN availablility:', chainer.cuda.cudnn_enabled)

畳み込みニューラルネットワークを定義します．ここでは，畳み込み層２層，全結合層３層から構成されるネットワークとします．１層目の畳み込み層は入力チャンネル数が１，出力する特徴マップ数が16，畳み込むフィルタサイズが3x3です．２層目の畳み込み層は入力チャネル数が16．出力する特徴マップ数が32，畳み込むフィルタサイズは同じく3x3です．１つ目の全結合層は入力ユニット数は不定とし，出力は1024としています．次の全結合層入力，出力共に1024，出力層は入力が1024，出力が10です．これらの各層の構成を\__init\__関数で定義します．
次に，\__call\__関数では，定義した層を接続して処理するように記述します．\__call\__関数の引数xは入力データです．それを\__init\__関数で定義したconv1に与え，その出力を活性化関数であるrelu関数に与えます．そして，その出力をmax_pooling_2dに与えて，プーリング処理結果をhとして出力します．hはconv2に与えられて畳み込み処理とプーリング処理を行います．そして，出力hをl1に与えて全結合層の処理を行います．最終的にl3の全結合層の処理を行った出力hを戻り値としています．


In [ ]:
class BottleNeck(chainer.Chain):

    def __init__(self, n_in, n_mid, n_out, stride=1, use_conv=False):
        w = chainer.initializers.HeNormal()
        super(BottleNeck, self).__init__()
        with self.init_scope():
            self.conv1 = L.Convolution2D(n_in, n_mid, 1, stride, 0, True, w)
            self.bn1 = L.BatchNormalization(n_mid)
            self.conv2 = L.Convolution2D(n_mid, n_mid, 3, 1, 1, True, w)
            self.bn2 = L.BatchNormalization(n_mid)
            self.conv3 = L.Convolution2D(n_mid, n_out, 1, 1, 0, True, w)
            self.bn3 = L.BatchNormalization(n_out)
            if use_conv:
                self.conv4 = L.Convolution2D(
                    n_in, n_out, 1, stride, 0, True, w)
                self.bn4 = L.BatchNormalization(n_out)
        self.use_conv = use_conv

    def __call__(self, x):
        h = F.relu(self.bn1(self.conv1(x)))
        h = F.relu(self.bn2(self.conv2(h)))
        h = self.bn3(self.conv3(h))
        return h + self.bn4(self.conv4(x)) if self.use_conv else h + x


class Block(chainer.ChainList):

    def __init__(self, n_in, n_mid, n_out, n_bottlenecks, stride=2):
        super(Block, self).__init__()
        self.add_link(BottleNeck(n_in, n_mid, n_out, stride, True))
        for _ in range(n_bottlenecks - 1):
            self.add_link(BottleNeck(n_out, n_mid, n_out))

    def __call__(self, x):
        for f in self:
            x = f(x)
        return x


class ResNet(chainer.Chain):

    def __init__(self, n_class=10, n_blocks=[3, 4, 6, 3]):
        super(ResNet, self).__init__()
        w = chainer.initializers.HeNormal()
        with self.init_scope():
            self.conv1 = L.Convolution2D(None, 64, 3, 1, 0, True, w)
            self.bn2 = L.BatchNormalization(64)
            self.res3 = Block(64, 64, 256, n_blocks[0], 1)
            self.res4 = Block(256, 128, 512, n_blocks[1], 2)
            self.res5 = Block(512, 256, 1024, n_blocks[2], 2)
            self.res6 = Block(1024, 512, 2048, n_blocks[3], 2)
            self.fc7 = L.Linear(None, n_class)

    def __call__(self, x):
        h = F.relu(self.bn2(self.conv1(x)))
        h = self.res3(h)
        h = self.res4(h)
        h = self.res5(h)
        h = self.res6(h)
        h = F.average_pooling_2d(h, h.shape[2:])
        h = self.fc7(h)
        return h


class ResNet50(ResNet):

    def __init__(self, n_class=10):
        super(ResNet50, self).__init__(n_class, [3, 4, 6, 3])

次に学習データを読み込みます．CIFAR10データセットは中規模な一般物体認識のデータセットであり，chainerでは CIFAR10
データセットを取得し，学習するためのフォーマットに変換してくれます．データセットには学習用とテスト用のデータに別れており，それぞれtrain_dataset, test_datasetとします．また，それらには画像データと教師ラベルがあり，それらをtrain_xとtrain_y，test_xとtest_yとします．

In [ ]:
#train_dataset, test_dataset = chainer.datasets.get_cifar10(scale=255.)
train_dataset, test_dataset = chainer.datasets.get_cifar10()
mean = np.mean([x for x, _ in train_dataset], axis=(0, 2, 3))
std = np.std([x for x, _ in train_dataset], axis=(0, 2, 3))
print (len(train_dataset), len(test_dataset))

In [ ]:
def transform(  inputs, mean, std, expand_ratio=1.0, crop_size=(32, 32), train=True):
    img, label = inputs
    img = img.copy()

    if train:
        # Random flip
        img = transforms.random_flip(img, x_random=True)
        # Random expand
        if expand_ratio > 1:
            img = transforms.random_expand(img, max_ratio=expand_ratio)
        # Random crop
        if tuple(crop_size) != (32, 32):
            img = transforms.random_crop(img, tuple(crop_size))

    return img, label


In [ ]:
train_transform = partial( transform, mean=mean, std=std, expand_ratio=1.2, crop_size=[28, 28], train=True)
valid_transform = partial(transform, mean=mean, std=std, train=False)

train_dataset = TransformDataset(train_dataset, train_transform)
test_dataset = TransformDataset(test_dataset, valid_transform)


畳み込みネットワークモデルを定義します．ここでは，GPUで学習を行うために，modelをGPUに送るto_gpu関数を利用しています．学習を行う際の最適化方法としてモーメンタムをもつ確率的勾配降下法（SGD）を利用します．SGDには学習率を指定します．その最適化方法に定義したネットワークモデルを与えます．

In [ ]:
gpu_id = 0 
xp = cuda.cupy
model = ResNet50()
model.to_gpu(gpu_id)

learnrate = 0.01
optimizer = chainer.optimizers.MomentumSGD(learnrate)
optimizer.setup(model)
optimizer.add_hook(chainer.optimizer.WeightDecay(5e-4))


１回の誤差を算出するデータ数（ミニバッチサイズ）を128，学習エポック数を100とします．MNISTの学習データサイズを取得し，１エポック内における更新回数を求めます．学習データは毎エポックでランダムに利用するため，numpyのpermutationという関数を利用します．各更新において，学習用データと教師データをそれぞれxとtとし，to_gpu関数でGPUに転送します．学習モデルにxを与えて各クラスの確率yを取得します．各クラスの確率yと教師ラベルtとの誤差をsoftmax coross entropy誤差関数で算出します．また，認識精度も算出します．そして，誤差をbackward関数で逆伝播し，ネットワークの更新を行います．

In [ ]:
batch_size = 128
epoch_num = 100

for epoch in range(epoch_num):
        dataset_x = []
        dataset_y =[]
        for train in train_dataset:
            dataset_x.append(train[0])
            dataset_y.append(train[1])
  
        train_x = xp.asarray(dataset_x, xp.float32)
        train_y = xp.asarray(dataset_y, xp.int32)
        print (train_x.shape, train_y.shape)
  
        train_data_num = train_x.shape[0]
        iter_one_epoch = int(train_x.shape[0]/batch_size)
        sum_loss = 0
        sum_accuracy = 0
        if (epoch+1) % 25 == 0 :
            optimizer.lr *= 0.5
        perm = xp.random.permutation(train_data_num)
        for i in range(0, train_data_num, batch_size):
                x = Variable(cuda.to_gpu(train_x[perm[i:i+batch_size]]))
                t = Variable(cuda.to_gpu(train_y[perm[i:i+batch_size]]))
                y = model(x)        
                model.zerograds()
                loss = F.softmax_cross_entropy(y, t)
                acc = F.accuracy(y, t)
                loss.backward()
                optimizer.update()
                sum_loss += loss.data
                sum_accuracy += acc.data
        print("epoch: {}, mean loss: {}, mean accuracy: {}".format(epoch+1, sum_loss*batch_size/train_data_num, sum_accuracy*batch_size/train_data_num))
        
        if (epoch+1) % 10 == 0:
            chainer.serializers.save_hdf5('caifr10_aug.npz', model, compression=6)

学習できたネットワークモデルを利用して評価を行います．

In [ ]:
cnt = 0
dataset_x = []
dataset_y =[]
for test in test_dataset:
    dataset_x.append(test[0])
    dataset_y.append(test[1])
test_x = xp.asarray(dataset_x, xp.float32)
test_y = xp.asarray(dataset_y, xp.int32)
print (test_x.shape, test_y.shape)
  
test_data_num = test_x.shape[0]
for i in range(0, test_data_num):
       x = Variable(cuda.to_gpu(test_x[i].reshape(1,3,32,32)))
       t = test_y[i]
       y = model(x)        
       y = np.argmax(y.data[0])
       if t == y:
           cnt += 1
print("test accuracy: {}".format(cnt/test_data_num))

In [ ]:
from IPython.display import display, Image

for data in test_dataset[:100]:
    bimg = data[0].copy()
    label = data[1]
    bimg *=255.
    img = bimg.transpose(1,2,0)
    #display_cv_image(img, '.png')
    decoded_bytes = cv2.imencode('.png', img)[1].tobytes()
    display(Image(data=decoded_bytes)) 

## 課題　
###以下の課題に取り組みましょう
1  ネットワーク構造を変えて実験しましょう

2  最適化の方法をAdam以外に変えて実験しましょう

3  エポック数やミニバッチサイズを変えて実験しましょう

4  GPUの有無による速度の差を比較しましょう
